<a href="https://colab.research.google.com/github/Shreyash54/HealthMisinfoDetection/blob/main/31_3_24_with10000_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM   WITH 73      ACCURACY   USING LARGER DATASET
REQUIRES CLEANSING AND HYPERPARAMETER TUNING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/cleaned-new-dataset.csv")
data.head()

,claim_id,statements,label
0,15661,$The money#@ the Clinton Foundation took from ...,False
1,9893,$Annual Mammograms$ May Have More FalsePositiv...,False
2,11358,Offers# Prostate $Cancer Patients High Cancer...,False
3,10166,Study Vaccine for Breast Ovarian Cancer Has Po...,True
4,11276,Some# appendicitis$ cases may not require emer...,True


In [ ]:
def summarize_text(text):
    # Tokenize the text into sentences
    sentences = text.split('.')

    # Calculate TF-IDF scores for the sentences
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Calculate cosine similarity between sentence vectors
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Summarize by selecting top-ranked sentences
    num_sentences_in_summary = 2
    summary_indices = similarity_matrix.argsort(axis=1)[:, -num_sentences_in_summary:]
    summary_indices = summary_indices.flatten()

    # Form the summary
    summary = ' '.join([sentences[i] for i in summary_indices])

    return summary

# Apply the summarization function to each row in the dataset
data['summary'] = data['statements'].apply(summarize_text)

def clean_text(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    x = re.sub('rt','',x)
    return x

data['summary'] = data['summary'].apply(clean_text)


In [ ]:
# Filter medical-related words
medical_words = [
    "doctor", "hospital", "nurse", "patient", "clinic", "medicine", "health", "diagnosis", "treatment",
    "prescription", "symptoms", "pharmacy", "surgery", "therapy", "vaccine", "emergency", "injury",
    "illness", "recovery", "consultation", "appointment", "laboratory", "examination", "operation",
    "medication", "disease", "disorder", "allergy", "cancer", "diabetes", "cardiology", "neurology",
    "psychiatry", "pediatrics", "gynecology", "ophthalmology", "orthopedics", "radiology", "anatomy",
    "physiology", "pharmacology", "pathology", "dermatology", "respiratory", "digestive", "endocrine",
    "genetic", "immune", "integumentary", "muscular", "nervous", "reproductive", "skeletal", "urinary",
    "vital signs", "blood pressure", "heart rate", "temperature", "pulse", "breathing rate", "BMI",
    "first aid", "virus", "bacteria", "infection", "antibiotics", "painkiller", "anesthesia",
    "bandage", "cast", "stethoscope", "syringe", "scalpel", "gauze", "IV", "x-ray", "ultrasound",
    "MRI", "CT scan", "blood test", "urine test", "vaccination", "healthcare", "wellness", "mental health",
    "rehabilitation", "public health", "epidemic", "pandemic", "quarantine", "hygiene", "nutrition",
    "exercise", "physical therapy", "occupational therapy", "alternative medicine", "homeopathy",
    "acupuncture", "chiropractic", "massage therapy", "herbal medicine", "holistic", "pain management"
]
filtered_df = data[data['summary'].str.contains('|'.join(medical_words), case=False, na=False)]


In [ ]:
data.columns

Index(['claim_id', 'statements', 'label', 'summary'], dtype='object')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_df['summary'])
sequences = tokenizer.texts_to_sequences(filtered_df['summary'])
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)


In [ ]:
# Prepare labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(filtered_df['label'])

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, labels, test_size=0.2)


In [ ]:
#xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, labels, test_size=0.2)

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1
embedding_size = 32


In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Print model summary
print(model.summary())


Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 389, 32)           1140448   
                                                                 
 lstm_17 (LSTM)              (None, 100)               53200     
                                                                 
 dense_13 (Dense)            (None, 1)                 101       
                                                                 
Total params: 1193749 (4.55 MB)
Trainable params: 1193749 (4.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
batch_size = 64
num_epochs = 10
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), batch_size=batch_size, epochs=num_epochs)



Epoch 1/10
92/92 [==============================] - 60s 628ms/step - loss: 0.5976 - accuracy: 0.6860 - val_loss: 0.5080 - val_accuracy: 0.7498
Epoch 2/10
92/92 [==============================] - 58s 629ms/step - loss: 0.3487 - accuracy: 0.8482 - val_loss: 0.5596 - val_accuracy: 0.7772
Epoch 3/10
92/92 [==============================] - 64s 694ms/step - loss: 0.1444 - accuracy: 0.9504 - val_loss: 0.6760 - val_accuracy: 0.7450
Epoch 4/10
92/92 [==============================] - 58s 632ms/step - loss: 0.0532 - accuracy: 0.9844 - val_loss: 0.8435 - val_accuracy: 0.7539
Epoch 5/10
92/92 [==============================] - 57s 620ms/step - loss: 0.0348 - accuracy: 0.9892 - val_loss: 0.7571 - val_accuracy: 0.7232
Epoch 6/10
92/92 [==============================] - 63s 682ms/step - loss: 0.0310 - accuracy: 0.9911 - val_loss: 0.8867 - val_accuracy: 0.7081
Epoch 7/10
92/92 [==============================] - 63s 689ms/step - loss: 0.0088 - accuracy: 0.9979 - val_loss: 1.1796 - val_accuracy: 0.7252

In [ ]:
loss, accuracy = model.evaluate(xtest, ytest)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

46/46 [==============================] - 4s 87ms/step - loss: 1.2179 - accuracy: 0.7252
Test Loss: 1.2179148197174072
Test Accuracy: 0.7252221703529358


In [ ]:
xtrain.shape , ytrain.shape , xtest.shape , ytest.shape

((5850, 389), (5850,), (1463, 389), (1463,))

In [ ]:
padded_sequences

array([[   0,    0,    0, ...,   19, 7870,  198],
       [   0,    0,    0, ...,  460,  977, 1810],
       [   0,    0,    0, ...,  347,   61,  882],
       ...,
       [   0,    0,    0, ...,  581,   58,   43],
       [   0,    0,    0, ...,    2,  392,  156],
       [   0,    0,    0, ...,  563,  780, 2203]], dtype=int32)

In [ ]:
labels

array([0, 0, 0, ..., 1, 1, 0])